In [6]:
rm -rf /root/.torch/iopath_cache/


In [7]:
mkdir -p /root/.torch/iopath_cache/s/57zjbwv6gh3srry/

Install Libraries

In [1]:
'''
Dependency Explanations:

sys:            Provides access to Python interpreter variables and functions, used here
                to check Python version

libreoffice:    Office suite used to convert DOCX files to PDF format for processing

poppler-utils:  PDF rendering library that powers the pdf2image package for PDF to
                image conversion

tesseract-ocr:  Open-source OCR engine that extracts text from images

torch:          PyTorch deep learning framework that provides tensor computation and
                neural networks

torchvision:    Computer vision library that works with PyTorch for image processing tasks

pdf2image:      Converts PDF documents to images using poppler under the hood

opencv-python:  Computer vision library for image processing operations

layoutparser:   Library for document layout analysis and segmentation

pytesseract:    Python wrapper for Google's Tesseract OCR engine

cython:         Language that makes writing C extensions for Python easier, required
                for detectron2

pycocotools:    Tools for working with the COCO dataset format, used by detectron2

fvcore:         Core libraries for computer vision research, used by detectron2

detectron2:     Facebook AI Research's object detection and segmentation framework
                used by layoutparser
'''


# Check Python version
import sys
print(f"Python version: {sys.version}")

# Install system dependencies
!apt-get update -qq
!apt-get install -y libreoffice poppler-utils tesseract-ocr -qq  # For DOCX to PDF, pdf2image, and OCR

# Clean up existing torch-related packages to avoid conflicts
#!pip uninstall torch torchaudio torchvision fastai layoutparser detectron2 fvcore -y --quiet

# Install latest Python dependencies
!pip install torch torchvision pdf2image opencv-python layoutparser pytesseract --quiet

# Install prerequisites for detectron2
!pip install cython pycocotools fvcore --quiet

# Install detectron2 from source (compatible with torch 2.6.0+cu124)
#!pip install 'git+https://github.com/facebookresearch/detectron2.git'

!pip install python-docx
!pip install --upgrade pillow
!sudo apt-get install tesseract-ocr-eng
#!pip install "detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.5#egg=detectron2"
!pip install -U 'git+https://github.com/facebookresearch/detectron2.git@ff53992b1985b63bd3262b5a36167098e3dada02'
!pip install "layoutparser[ocr]"

!pip install sentence-transformers faiss-cpu fastapi uvicorn pyngrok

Python version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.
  Cloning https://github.com/facebookresearch/detectron2.git (to revision ff53992b1985b63bd3262b5a36167098e3dada02) to /tmp/pip-req-build-ew6a2yiy
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-ew6a2yiy
  Running command git rev-parse -q --verify 'sha^ff53992b1985b63bd3262b5a36167098e3dada02'
  Running command git fetch -q https://github.com/facebookresearch/detectron2.git ff53992b1985b63bd3262b5a36167098e3dada02
  Running co

In [49]:
!mv /root/.torch/iopath_cache/s/57zjbwv6gh3srry/model_final.pth?dl=1 /root/.torch/iopath_cache/s/57zjbwv6gh3srry/model_final.pth

Download Model, Convert word->pdf->png, extract and classify content, store metadata and index

In [26]:
# Verify installations
import torch
import torchvision
print(f"Torch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")

import layoutparser as lp
print(f"LayoutParser version: {lp.__version__}")

import detectron2
print(f"Detectron2 version: {detectron2.__version__}")

import os
import subprocess
from datetime import datetime
import json
from pathlib import Path
from pdf2image import convert_from_path
import cv2
import pytesseract
import numpy as np
from PIL import Image
from docx import Document
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx.shared import Pt
import re

# Mount Google Drive
from google.colab import drive,userdata
drive.mount('/content/drive')



# Import new libraries
from sentence_transformers import SentenceTransformer
import faiss

# Set up directories
SOURCE_DIR = "/content/drive/My Drive/lifesciences/training_documents"
PDF_STAGING_DIR = "/content/drive/My Drive/lifesciences/staging/pdf"
IMG_STAGING_DIR = "/content/drive/My Drive/lifesciences/staging/images"
METADATA_FILE = "/content/drive/My Drive/lifesciences/metadata.json"
ANNOTATED_DIR = "/content/drive/My Drive/lifesciences/staging/annotated"
FAISS_INDEX_PATH = "/content/drive/My Drive/lifesciences/faiss_index.index"

os.makedirs(SOURCE_DIR, exist_ok=True)
os.makedirs(PDF_STAGING_DIR, exist_ok=True)
os.makedirs(IMG_STAGING_DIR, exist_ok=True)
os.makedirs(ANNOTATED_DIR, exist_ok=True)

# Increase PIL's image size limit
Image.MAX_IMAGE_PIXELS = 100000000

# Initialize ID sequence
id_seq = 1

# Initialize SentenceTransformer
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
print("Loaded SentenceTransformer: all-MiniLM-L6-v2")

# Initialize FAISS index
dimension = 384  # all-MiniLM-L6-v2 output dimension
faiss_index = faiss.IndexFlatL2(dimension)
metadata_entries = []  # To store JSON entries for mapping

# Load PubLayNet model
MODEL_CONFIG = "lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config"
model = lp.Detectron2LayoutModel(config_path=MODEL_CONFIG,
                                model_path='/root/.torch/iopath_cache/s/57zjbwv6gh3srry/model_final.pth',
                                 extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.2],
                                 label_map={0: "Text", 1: "Title", 2: "List", 3: "Table", 4: "Figure"})

print("Loading PubLayNet model...")

# Initialize Tesseract OCR
ocr_agent = lp.TesseractAgent(languages='eng', config='--psm 6 --oem 1')

# Color map for visualization (BGR format for OpenCV)
COLORS = {
    "title": (255, 0, 0),    # Blue
    "text": (0, 255, 0),     # Green
    "list": (0, 0, 255),     # Red
    "table": (255, 255, 0),  # Cyan
    "figure": (255, 0, 255), # Magenta
}

def add_table_border(table):
    """Add thick borders to a table, including internal gridlines."""
    tbl = table._tbl
    tblBorders = OxmlElement('w:tblBorders')
    for border_name in ['top', 'left', 'bottom', 'right', 'insideH', 'insideV']:
        border = OxmlElement(f'w:{border_name}')
        border.set(qn('w:val'), 'single')
        border.set(qn('w:sz'), '12')
        border.set(qn('w:color'), '000000')
        tblBorders.append(border)
    tbl.append(tblBorders)
    for row in table.rows:
        for cell in row.cells:
            tc = cell._tc
            tcBorders = OxmlElement('w:tcBorders')
            for border_name in ['top', 'left', 'bottom', 'right']:
                border = OxmlElement(f'w:{border_name}')
                border.set(qn('w:val'), 'single')
                border.set(qn('w:sz'), '8')
                border.set(qn('w:color'), '000000')
                tcBorders.append(border)
            tc.append(tcBorders)

def adjust_docx_titles_and_spacing(docx_path, temp_docx_path, min_spacing=40):
    """Detect titles, convert them to all caps, and adjust spacing."""
    doc = Document(docx_path)
    for para in doc.paragraphs:
        is_title = False
        if para.style and "Heading" in para.style.name:
            is_title = True
        elif para.runs and all(run.bold for run in para.runs if run.text.strip()):
            is_title = True
        if is_title and para.text.strip():
            original_text = para.text
            para.clear()
            new_run = para.add_run(original_text.upper())
            new_run.bold = True
            new_run.underline = True
            print(f"Formatted title: '{original_text}' → '{new_run.text}'")
    for table in doc.tables:
        add_table_border(table)
    doc.save(temp_docx_path)
    return temp_docx_path

def convert_docx_to_pdf(docx_path, output_dir):
    """Convert DOCX to PDF using LibreOffice with enforced vertical gaps."""
    if not os.path.exists(docx_path):
        raise FileNotFoundError(f"Source DOCX file not found: {docx_path}")
    final_output_path = os.path.join(output_dir, f"{Path(docx_path).stem}.pdf")
    temp_docx_path = os.path.join(output_dir, f"{Path(docx_path).stem}_temp.docx")
    adjusted_docx = adjust_docx_titles_and_spacing(docx_path, temp_docx_path, min_spacing=40)
    print(f"Converting {adjusted_docx} to {final_output_path}...")
    result = subprocess.run([
        'libreoffice',
        '--headless',
        '--convert-to',
        'pdf',
        adjusted_docx,
        '--outdir',
        output_dir
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode != 0:
        raise subprocess.CalledProcessError(
            result.returncode,
            result.args,
            output=result.stdout,
            stderr=result.stderr
        )
    print(f"LibreOffice stdout: {result.stdout}")
    if result.stderr:
        print(f"LibreOffice stderr: {result.stderr}")
    temp_pdf_path = os.path.join(output_dir, f"{Path(adjusted_docx).stem}.pdf")
    if os.path.exists(temp_pdf_path):
        os.rename(temp_pdf_path, final_output_path)
        print(f"Renamed {temp_pdf_path} to {final_output_path}")
    if not os.path.exists(final_output_path):
        raise FileNotFoundError(f"Final PDF file was not created: {final_output_path}")
    if os.path.exists(temp_docx_path):
        os.remove(temp_docx_path)
    return final_output_path

def convert_pdf_to_png(pdf_path, output_dir):
    """Convert PDF pages to PNG images."""
    pdf_name = Path(pdf_path).stem
    try:
        images = convert_from_path(pdf_path, dpi=1000)
    except Exception as e:
        print(f"Failed to convert PDF to images: {str(e)}")
        return []
    if not images:
        print(f"No images generated from {pdf_path}")
        return []
    image_paths = []
    for i, image in enumerate(images):
        img_array = np.array(image)
        if img_array.size == 0:
            print(f"Image {i+1} is empty for {pdf_name}")
            continue
        output_path = os.path.join(output_dir, f"{pdf_name}_page_{i + 1}.png")
        image.save(output_path, "PNG", quality=95)
        image_paths.append(output_path)
        print(f"Saved image: {output_path}")
    return image_paths

def is_overlapping(rect1, rect2):
    """Check if two rectangles overlap."""
    x1, y1, x2, y2 = rect1
    x3, y3, x4, y4 = rect2
    if x2 <= x3 or x4 <= x1 or y2 <= y3 or y4 <= y1:
        return False
    return True

def process_layout(image_path):
    """Process PNG using LayoutParser and extract elements with visualization, skipping overlapping text."""
    global id_seq
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Failed to load image: {image_path}")
    annotated_image = image.copy()
    layout = model.detect(image)
    sorted_layout = sorted(layout, key=lambda x: x.coordinates[1])
    results = []
    current_section = "No Title"
    structural_regions = []
    for block in sorted_layout:
        block_type = block.type.lower()
        if block_type in ["table", "list", "title"]:
            structural_regions.append({
                "type": block_type,
                "coordinates": list(map(int, block.coordinates))
            })
    for block in sorted_layout:
        x1, y1, x2, y2 = map(int, block.coordinates)
        block_type = block.type.lower()
        print(f"Detected Type: {block_type}, Coordinates: ({x1}, {y1}, {x2}, {y2})")
        if block_type == "text":
            text_coords = (x1, y1, x2, y2)
            overlaps = False
            for region in structural_regions:
                region_coords = region["coordinates"]
                if is_overlapping(text_coords, region_coords):
                    overlaps = True
                    print(f"Skipping overlapping text at ({x1}, {y1}, {x2}, {y2}) with {region['type']}")
                    break
            if overlaps:
                continue
        if block_type in ["list", "table"]:
            cropped_image = image[y1:y2, x1:x2]
            content = ocr_agent.detect(cropped_image) or ""
            if block_type == "list":
                lines = content.split('\n')
                cleaned_lines = [re.sub(r'[^a-zA-Z0-9\s]', '', line.strip()) for line in lines if line.strip()]
                content = '\n'.join(cleaned_lines) if cleaned_lines else ""
            elif block_type == "table":
                rows = content.split('\n')
                cleaned_rows = []
                for row in rows:
                    if row.strip():
                        columns = row.split()
                        cleaned_row = '-'.join(col.strip() for col in columns if col.strip())
                        cleaned_rows.append(cleaned_row.replace('\n', ' '))
                content = '\n'.join(cleaned_rows) if cleaned_rows else "Table Content"
        else:
            cropped_image = image[y1:y2, x1:x2]
            if block_type == "title":
                content = ocr_agent.detect(cropped_image) or "Untitled"
                current_section = content.lower()
            else:
                content = ocr_agent.detect(cropped_image) or ""
        cv2.rectangle(annotated_image, (x1, y1), (x2, y2), COLORS[block_type], 2)
        cv2.putText(annotated_image, block_type, (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, COLORS[block_type], 2)
        result = {
            "id": id_seq,
            "urs": Path(image_path).stem,
            "section": current_section,
            "type": block_type,
            "content": content.strip(),
            "coordinates": list(block.coordinates),
            "created_at": datetime.now().isoformat(),
            "source_file": image_path
        }
        results.append(result)
        id_seq += 1
    annotated_path = os.path.join(ANNOTATED_DIR, Path(image_path).name.replace(".png", "_annotated.png"))
    cv2.imwrite(annotated_path, annotated_image)
    print(f"Saved annotated image: {annotated_path}")
    append_to_metadata(results)
    return results

def append_to_metadata(data):
    """Append results to metadata.json"""
    if not os.path.exists(METADATA_FILE):
        with open(METADATA_FILE, 'w') as f:
            json.dump([], f)
    with open(METADATA_FILE, 'r') as f:
        existing_data = json.load(f)
    existing_data.extend(data)
    with open(METADATA_FILE, 'w') as f:
        json.dump(existing_data, f, indent=2)

def process_file(file_path):
    """Process a single file through all steps"""
    file_ext = Path(file_path).suffix.lower()
    pdf_path = file_path
    if file_ext == '.docx':
        try:
            pdf_path = convert_docx_to_pdf(file_path, PDF_STAGING_DIR)
        except Exception as e:
            print(f"Error converting DOCX to PDF for {file_path}: {str(e)}")
            return
    if not os.path.exists(pdf_path):
        print(f"PDF file not found: {pdf_path}. Skipping...")
        return
    try:
        png_paths = convert_pdf_to_png(pdf_path, IMG_STAGING_DIR)
    except Exception as e:
        print(f"Error converting PDF to PNG for {pdf_path}: {str(e)}")
        return
    all_results = []
    for png_path in png_paths:
        try:
            results = process_layout(png_path)
            all_results.extend(results)
        except Exception as e:
            print(f"Error processing layout for {png_path}: {str(e)}")
    append_to_metadata(all_results)
    print(f"Processed: {Path(file_path).name}, Added {len(all_results)} entries")
    return all_results

def build_faiss_index():
    """Build FAISS index from metadata.json with URS and section indexing"""
    global faiss_index, metadata_entries

    if not os.path.exists(METADATA_FILE):
        print(f"Metadata file {METADATA_FILE} not found. Run processing first.")
        return

    with open(METADATA_FILE, 'r') as f:
        metadata = json.load(f)

    if not metadata:
        print("No entries in metadata to index.")
        return

    # Prepare data for FAISS indexing
    combined_texts = []
    metadata_entries = []

    for entry in metadata:
        urs = entry.get("urs", "").strip()
        section = entry.get("section", "").strip()
        content = entry.get("content", "").strip()
        content_type = entry.get("type", "").strip()
        if content:
            # Create a combined text for better indexing
            combined_text = f"{content_type} {urs} {section} {content} "
            combined_texts.append(combined_text)
            print(f"Combined Text: {combined_text}")


    # Generate embeddings
    embeddings = sentence_model.encode(combined_texts, convert_to_numpy=True)
    print(f"Generated embeddings for {len(combined_texts)} entries")
    # Reinitialize FAISS index
    dimension = embeddings.shape[1]
    print(f"Dimension of embeddings: {dimension}")
    faiss_index = faiss.IndexFlatL2(dimension)
    faiss_index.add(embeddings)

    # Save index
    faiss.write_index(faiss_index, FAISS_INDEX_PATH)
    print(f"FAISS index built with {len(combined_texts)} entries and saved to {FAISS_INDEX_PATH}")


def process_and_index():
    """Process files and build FAISS index"""
    for filename in sorted(os.listdir(SOURCE_DIR)):
        file_path = os.path.join(SOURCE_DIR, filename)
        if os.path.isfile(file_path) and filename.lower().endswith(('.docx', '.pdf')):
            print(f"Starting: {filename}")
            process_file(file_path)
    print(f"Processing complete. Total entries: {id_seq - 1}")
    build_faiss_index()



if __name__ == "__main__":
    !nvidia-smi | grep "CUDA Version" || echo "nvidia-smi not found"
    print("Processing files from:", SOURCE_DIR)
    process_and_index()


Torch version: 2.5.1+cu124
Torchvision version: 0.20.1+cu124
LayoutParser version: 0.3.4
Detectron2 version: 0.6
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded SentenceTransformer: all-MiniLM-L6-v2


/usr/local/lib/python3.11/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

Loading PubLayNet model...
/bin/bash: line 1: nvidia-smi: command not found
nvidia-smi not found
Processing files from: /content/drive/My Drive/lifesciences/training_documents
Starting: Pharma_URS_Enhanced.docx
Converting /content/drive/My Drive/lifesciences/staging/pdf/Pharma_URS_Enhanced_temp.docx to /content/drive/My Drive/lifesciences/staging/pdf/Pharma_URS_Enhanced.pdf...
LibreOffice stdout: convert /content/drive/My Drive/lifesciences/staging/pdf/Pharma_URS_Enhanced_temp.docx -> /content/drive/My Drive/lifesciences/staging/pdf/Pharma_URS_Enhanced_temp.pdf using filter : writer_pdf_Export

Renamed /content/drive/My Drive/lifesciences/staging/pdf/Pharma_URS_Enhanced_temp.pdf to /content/drive/My Drive/lifesciences/staging/pdf/Pharma_URS_Enhanced.pdf
Saved image: /content/drive/My Drive/lifesciences/staging/images/Pharma_URS_Enhanced_page_1.png
Saved image: /content/drive/My Drive/lifesciences/staging/images/Pharma_URS_Enhanced_page_2.png
Detected Type: title, Coordinates: (1292, 10

Query FAISS Index

In [39]:
import faiss
from fastapi import FastAPI
import os
import json
from sentence_transformers import SentenceTransformer

FAISS_INDEX_PATH = "/content/drive/My Drive/lifesciences/faiss_index.index"
METADATA_FILE="/content/drive/My Drive/lifesciences/metadata.json"

# Initialize SentenceTransformer
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
print("Loaded SentenceTransformer: all-MiniLM-L6-v2")

def query_faiss_index(urs_name, section_name, content_type, top_k=10):
    """Query FAISS index with urs_name and section_name, return matching JSON entries"""
    if os.path.exists(FAISS_INDEX_PATH):
        faiss_index = faiss.read_index(FAISS_INDEX_PATH)
        with open(METADATA_FILE, 'r') as f:
            global metadata_entries
            metadata_entries = json.load(f)
    else:
        print("FAISS index is empty and no saved index found.")
        return []
    query = f"{content_type} {urs_name} {section_name}"
    print(query)
    query_embedding = sentence_model.encode([query], convert_to_numpy=True)
    distances, indices = faiss_index.search(query_embedding, top_k)
    print(f"distances : {distances}, indices:{indices} number_of_metdata_entries: {len(metadata_entries)}")
    results = [metadata_entries[idx] for idx in indices[0] if idx < len(metadata_entries)]
    print(results)
    return results

# FastAPI app
app = FastAPI()

@app.get("/search")
async def search_documents(urs_name: str, section_name: str, content_type :str, top_k: int = 5):
    """Search FAISS index by urs_name and section_name"""
    results = query_faiss_index(urs_name, section_name,content_type, top_k)
    return {"results": results}


Loaded SentenceTransformer: all-MiniLM-L6-v2


Run UVICORN server and access via NGROK Tunnel

In [ ]:

import uvicorn
from pyngrok import ngrok
import nest_asyncio

# Mount Google Drive
from google.colab import drive,userdata
drive.mount('/content/drive')

def run_fastapi_with_ngrok():
    """Run FastAPI app and expose via ngrok"""
    nest_asyncio.apply()  # Allow running uvicorn in Colab
    ngrok.set_auth_token(userdata.get('ngrok_auth_token'))  # Replace with your ngrok auth token
    public_url = ngrok.connect(8000)
    print(f"Public URL: {public_url}")
    uvicorn.run(app, host="0.0.0.0", port=8000)
run_fastapi_with_ngrok()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Public URL: NgrokTunnel: "https://100e-34-106-83-184.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [27520]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


list Pharma_URS_Enhanced_page_2 core functionalities
distances : [[0.1613935  0.1613935  0.42887896 0.42887896 0.4900623 ]], indices:[[16 40  6 30 15]] number_of_metdata_entries: 48
[{'id': 17, 'urs': 'Pharma_URS_Enhanced_page_2', 'section': '8 core functionalities\n\x0c', 'type': 'title', 'content': '8 Core Functionalities', 'coordinates': [1322.971435546875, 3069.854248046875, 5001.3017578125, 3434.02587890625], 'created_at': '2025-03-10T10:54:21.031137', 'source_file': '/content/drive/My Drive/lifesciences/staging/images/Pharma_URS_Enhanced_page_2.png'}, {'id': 17, 'urs': 'Pharma_URS_Enhanced_page_2', 'section': '8 core functionalities\n\x0c', 'type': 'title', 'content': '8 Core Functionalities', 'coordinates': [1322.971435546875, 3069.854248046875, 5001.3017578125, 3434.02587890625], 'created_at': '2025-03-10T10:54:21.031137', 'source_file': '/content/drive/My Drive/lifesciences/staging/images/Pharma_URS_Enhanced_page_2.png'}, {'id': 7, 'urs': 'Pharma_URS_Enhanced_page_1', 'section

In [29]:
!python --version

Python 3.11.11


In [30]:
ls -lart /root/.torch/iopath_cache/s/57zjbwv6gh3srry/


total 1672432
-rw-r--r-- 1 root root         0 Mar 10 09:46 'model_final.pth?dl=1.lock'
drwxr-xr-x 4 root root      4096 Mar 10 09:46  ../
-rw-r--r-- 1 root root 856276388 Mar 10 09:50  model_final.pth
-rw-r--r-- 1 root root 856276388 Mar 10 09:52 'model_final.pth?dl=1'
drwxr-xr-x 2 root root      4096 Mar 10 09:52  ./


In [31]:
ls -lart /root/.torch/iopath_cache/s/57zjbwv6gh3srry

total 1672432
-rw-r--r-- 1 root root         0 Mar 10 09:46 'model_final.pth?dl=1.lock'
drwxr-xr-x 4 root root      4096 Mar 10 09:46  ../
-rw-r--r-- 1 root root 856276388 Mar 10 09:50  model_final.pth
-rw-r--r-- 1 root root 856276388 Mar 10 09:52 'model_final.pth?dl=1'
drwxr-xr-x 2 root root      4096 Mar 10 09:52  ./
